In [1]:
# Imports

import plotly
import optuna
import mlflow
import dagshub
import mlflow.sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# Setting up DagsHub

dagshub.init(repo_owner='SushrutGaikwad', repo_name='youtube-comments-analyzer', mlflow=True)

Accessing as SushrutGaikwad

Initialized MLflow to track repo "SushrutGaikwad/youtube-comments-analyzer"

Repository SushrutGaikwad/youtube-comments-analyzer initialized!

# Data

In [3]:
PREPROCESSED_DATA_PATH = "../data/processed/reddit_preprocessed.csv"
df = pd.read_csv(PREPROCESSED_DATA_PATH)
df.dropna(subset=["clean_comment"], inplace=True)
df.shape

(36662, 2)

# Running the experiment

In [4]:
# Setting experiment name

mlflow.set_experiment("Exp 5: Detailed hyperparameter tuning")

<Experiment: artifact_location='mlflow-artifacts:/50f3ef58d8344cd3bd0def333af39f2d', creation_time=1749058038347, experiment_id='6', last_update_time=1749064965442, lifecycle_stage='active', name='Exp 5: Detailed hyperparameter tuning', tags={}>

## Preprocessing

In [5]:
# Remapping class labels from {-1, 0, 1} to {2, 0, 1}
mapping = {
    -1: 2,
    0: 0,
    1: 1
}
df["category"] = df["category"].map(mapping)

# Removing missing sentiments
df.dropna(subset=["category"], inplace=True)

# Feature engineering
ngram_range = (1, 2)
max_features = 1000
vectorizer = CountVectorizer(
    ngram_range=ngram_range,
    max_features=max_features
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_comment"],
    df["category"],
    test_size=0.2,
    random_state=42,
    stratify=df["category"]
)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Undersampling
rus = RandomUnderSampler(random_state=42)
X_train_vectorized, y_train = rus.fit_resample(
    X_train_vectorized,
    y_train
)

## Running the experiment

In [6]:
# Function to log results to MLFlow
def log_to_mlflow(
    model_name,
    model,
    X_train,
    X_test,
    y_train,
    y_test,
    params,
    trial_number
):
    with mlflow.start_run():
        # Tags
        mlflow.set_tag(
            "mlflow.runName", f"Trial_{trial_number}_{model_name}_undersampling_BoW_1000_bigrams"
        )
        mlflow.set_tag("experiment_type", "ML_models_comparison")
        
        # Logging model name as a parameter
        mlflow.log_param("model_name", model_name)
        
        # Logging hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)
        
        # Initializing and training the model
        model.fit(X_train, y_train)
        
        # Making predictions on the test set and logging metrics
        y_pred = model.predict(X_test)
        
        # Logging accuracy
        accuracy = accuracy_score(
            y_true=y_test,
            y_pred=y_pred
        )
        mlflow.log_metric("accuracy", accuracy)
        
        # Logging classification report metrics
        classification_rep = classification_report(
            y_true=y_test,
            y_pred=y_pred,
            output_dict=True
        )
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}: {metric}", value)
        
        # Logging the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")
        
        return accuracy

In [7]:
def objective(trial):
    # Hyperparameter space
    C = trial.suggest_float("C", 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical("penalty", ["l2", "l1"])
    solver = trial.suggest_categorical("solver", ["liblinear", "saga"])
    
    params = {
        "C": C,
        "penalty": penalty,
        "solver": solver
    }
    
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        max_iter=1000,
        multi_class="ovr",
        random_state=42
    )
    
    # Logging each trial as a separate run
    accuracy = log_to_mlflow(
        model_name="LogisticRegression",
        model=model,
        X_train=X_train_vectorized,
        X_test=X_test_vectorized,
        y_train=y_train,
        y_test=y_test,
        params=params,
        trial_number=trial.number
    )
    
    return accuracy

In [8]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2025-06-05 05:19:05,450] A new study created in memory with name: no-name-bf25100f-3518-442c-8fa7-4f1082bf8cb1
d:\projects\DSMP2_p2\youtube-comments-analyzer\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
2025/06/05 05:19:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_0_LogisticRegression_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/57f017b1c03a41ebb9b27ab004f43e34
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 05:19:32,276] Trial 0 finished with value: 0.775671621437338 and parameters: {'C': 3.8419050827736068, 'penalty': 'l2', 'solver': 'liblinear'}. Best is trial 0 with value: 0.775671621437338.
d:\projects\DSMP2_p2\youtube-comments-analyzer\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
2025/06/05 05:20:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_1_LogisticRegression_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/808a80bae5064cb1bafd66b3eb8b7515
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 05:20:09,015] Trial 1 finished with value: 0.7856266193917906 and parameters: {'C': 0.41999867426830273, 'penalty': 'l1', 'solver': 'liblinear'}. Best is trial 1 with value: 0.7856266193917906.
d:\projects\DSMP2_p2\youtube-comments-analyzer\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
2025/06/05 05:20:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

[I 2025-06-05 05:51:30,015] Trial 47 finished with value: 0.7837174417018955 and parameters: {'C': 0.7981538505808767, 'penalty': 'l1', 'solver': 'liblinear'}. Best is trial 21 with value: 0.7864448384017455.
d:\projects\DSMP2_p2\youtube-comments-analyzer\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
2025/06/05 05:52:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_48_LogisticRegression_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/76393f9a223642b9a73413dcc46f6b23
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 05:52:07,384] Trial 48 finished with value: 0.7147143051956907 and parameters: {'C': 0.0440512752400271, 'penalty': 'l1', 'solver': 'saga'}. Best is trial 21 with value: 0.7864448384017455.
d:\projects\DSMP2_p2\youtube-comments-analyzer\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
2025/06/05 05:52:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_49_LogisticRegression_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/c3d16eb2f513497db9522ef0738b8143
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 05:52:44,389] Trial 49 finished with value: 0.7858993590617755 and parameters: {'C': 0.3407772951905502, 'penalty': 'l1', 'solver': 'liblinear'}. Best is trial 21 with value: 0.7864448384017455.


In [9]:
# Getting the best parameters
best_params = study.best_params
best_model = LogisticRegression(
    C=best_params["C"],
    penalty=best_params["penalty"],
    solver=best_params["solver"],
    max_iter=1000,
    multi_class="ovr",
    random_state=42
)

# Logging the best model
log_to_mlflow(
    model_name="LogisticRegression",
    model=best_model,
    X_train=X_train_vectorized,
    X_test=X_test_vectorized,
    y_train=y_train,
    y_test=y_test,
    params=best_params,
    trial_number="best"
)

d:\projects\DSMP2_p2\youtube-comments-analyzer\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
2025/06/05 05:53:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_best_LogisticRegression_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/489b50bce8cd4f5b8705486f9e879d34
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


0.7864448384017455

In [10]:
# Plotting parameter importance
optuna.visualization.plot_param_importances(study).show()

# Plotting optimization history
optuna.visualization.plot_optimization_history(study).show()